In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
# CONFIGURATION

DATA_ROOT = "/kaggle/input/plant-disease-dataset/Dataset_Final_V2_Split"  # change to your dataset path
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR   = os.path.join(DATA_ROOT, "val")
TEST_DIR  = os.path.join(DATA_ROOT, "test")

BATCH_SIZE = 128
IMG_SIZE = 224
EPOCHS = 50
LR = 3e-5
WEIGHT_DECAY = 1e-4
PATIENCE = 5
NUM_WORKERS = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAME = "vit_tiny_patch16_224"  
USE_PRETRAINED = True
OUTPUT_DIR = "/kaggle/working"

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
# DATASETS & DATALOADERS

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    normalize
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform)
valid_dataset = datasets.ImageFolder(VAL_DIR, transform=transform)
test_dataset  = datasets.ImageFolder(TEST_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

class_names = train_dataset.classes
num_classes = len(class_names)

print(f"✅ Data loaded: {len(train_dataset)} train, {len(valid_dataset)} valid, {len(test_dataset)} test images")
print(f"Classes: {num_classes} → {class_names}")


In [ ]:

# MODEL

print(f"\n🔧 Creating model: {MODEL_NAME}")
model = timm.create_model(MODEL_NAME, pretrained=USE_PRETRAINED, num_classes=num_classes)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

In [ ]:

# TRAINING LOOP

best_val_acc = 0.0
early_stop_counter = 0
history = []

def accuracy(output, target):
    _, preds = torch.max(output, 1)
    return (preds == target).sum().item()

for epoch in range(1, EPOCHS + 1):
    start_time = time.time()
    model.train()

    train_loss, train_correct, train_total = 0, 0, 0
    val_loss, val_correct, val_total = 0, 0, 0

    # ---- Training ----
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [Train]", leave=False)
    for inputs, labels in train_bar:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * inputs.size(0)
        train_correct += accuracy(outputs, labels)
        train_total += labels.size(0)
        progress = 100 * train_total / len(train_loader.dataset)
        train_bar.set_postfix(loss=loss.item(), progress=f"{progress:.1f}%")

    avg_train_loss = train_loss / train_total
    train_acc = 100 * train_correct / train_total
    # ---- Validation ----
    model.eval()
    with torch.no_grad():
        val_bar = tqdm(valid_loader, desc=f"Epoch {epoch}/{EPOCHS} [Val]", leave=False)
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += accuracy(outputs, labels)
            val_total += labels.size(0)
            progress = 100 * val_total / len(valid_loader.dataset)
            val_bar.set_postfix(loss=loss.item(), progress=f"{progress:.1f}%")

    avg_val_loss = val_loss / val_total
    val_acc = 100 * val_correct / val_total

    scheduler.step()
    epoch_time = time.time() - start_time

    print(f"Epoch {epoch:03}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}% | Time: {epoch_time:.1f}s")

    history.append({
        "Epoch": epoch,
        "Train Loss": avg_train_loss,
        "Train Acc": train_acc,
        "Val Loss": avg_val_loss,
        "Val Acc": val_acc,
        "Time (s)": epoch_time
    })
    pd.DataFrame(history).to_csv(os.path.join(OUTPUT_DIR, "vitv2_history.csv"), index=False)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_vitv2_model.pth"))
        print(f"🟢 Saved new best model (Val Acc: {best_val_acc:.2f}%)")
    else:
        early_stop_counter += 1
        print(f"⏸ Early stop counter: {early_stop_counter}/{PATIENCE}")
        if early_stop_counter >= PATIENCE:
            print("⚠️ Early stopping triggered.")
            break



In [ ]:
# TEST EVALUATION

print("\n🔍 Evaluating best model on test set...")
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "best_vitv2_model.pth")))
model.eval()

test_loss, test_correct, test_total = 0, 0, 0
all_labels, all_preds = [], []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

test_loss /= test_total
test_acc = 100 * test_correct / test_total
print(f"✅ Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")


In [ ]:

# CONFUSION MATRIX & REPORT

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(cmap="Blues", ax=ax, xticks_rotation=90)
plt.title("Confusion Matrix - ViT v2")
plt.show()

print("\n📊 Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))


In [ ]:
# TRAINING CURVES

hist_df = pd.read_csv(os.path.join(OUTPUT_DIR, "vitv2_history.csv"))

plt.figure(figsize=(10,5))
plt.plot(hist_df["Epoch"], hist_df["Train Acc"], label="Train Acc")
plt.plot(hist_df["Epoch"], hist_df["Val Acc"], label="Val Acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy (%)")
plt.title("Training & Validation Accuracy (ViT v2)")
plt.legend(); plt.grid(); plt.show()

plt.figure(figsize=(10,5))
plt.plot(hist_df["Epoch"], hist_df["Train Loss"], label="Train Loss")
plt.plot(hist_df["Epoch"], hist_df["Val Loss"], label="Val Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.title("Training & Validation Loss (ViT v2)")
plt.legend(); plt.grid(); plt.show()